<a href="https://colab.research.google.com/github/sio-co2o2/keelingcurve_notebooks/blob/main/notebooks/create_graphic_mlo_two_years_keelingcurve.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Create a graphic appearing on the Keeling Curve website
### **Graphic: CO<sub>2</sub> record at Mauna Loa, Hawaii for the last 2 years**

#### This notebook creates customizable PDF and PNG images of a graphic appearing on the front page of [keelingcurve.ucsd.edu](keelingcurve.ucsd.edu).

For an overview of the Keeling Curve notebooks, data and graphics licenses, and how to use the notebooks, see [notebooks overview](https://colab.research.google.com/github/sio-co2o2/keelingcurve_notebooks/blob/main/notebooks/overview_of_notebooks_keelingcurve.ipynb).

**To use the notebook**, first run all the cells. In Google Colab, select from the top menu and select Runtime -> Run all. Then the buttons will appear to click on which creates and downloads the graphics.

Direct questions to [webmaster-co2o2-sio@ucsd.edu](mailto:webmaster-co2o2-sio@ucsd.edu)

---

### [Go to creating the graphic section](#button-to-create-and-save-the-graphic)

---

### Sample of the graphic produced by this notebook

![title](https://github.com/sio-co2o2/keelingcurve_notebooks/blob/main/images/sample_graphics/mlo_two_years.png?raw=1)

## **Notebook Code**

### Import python packages

In [1]:
import functools
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta
import pathlib
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
from matplotlib import ticker
from matplotlib.path import Path
import matplotlib.colors as mcolors
from matplotlib.patches import Polygon
import matplotlib.dates as mdates
import matplotlib.lines as mlines
from scipy.interpolate import UnivariateSpline
from scipy import interpolate
import matplotlib.transforms
from matplotlib import animation
from ipywidgets import widgets
from IPython.display import HTML, display
from base64 import b64encode
from PIL import Image
import urllib
import os

%matplotlib inline

In [2]:
# This package is used to convert a vector svg into a png

try:
  from cairosvg import svg2png
except:
  ! pip install cairosvg
  from cairosvg import svg2png

In [3]:
# This import is to enable Google Colab to save files ane then download them

# This import does not exist unless the notebook is run in Google Colab
# Put in a try except block if user wants to use notebook off of Google Colab

try:
    from google.colab import files
except:
    pass

### Set directories and file names

In [4]:
# Read in data from github repository
data_file = 'https://raw.githubusercontent.com/sio-co2o2/keelingcurve_notebooks/main/preliminary_data/mlo/mlo_two_year_span.csv'

logo_file = 'https://github.com/sio-co2o2/keelingcurve_notebooks/raw/main/images/logos/ucsd_sio_logo.svg'

plot_dir = pathlib.Path('./plots')
plot_dir.mkdir(exist_ok=True)

plot_basename = 'mlo_two_years'

pdf_file = plot_dir / f"{plot_basename}.pdf"
png_file = plot_dir / f"{plot_basename}.png"

## **Load in data and prepare it for plotting**

### Load in MLO record

In [5]:
df = pd.read_csv(data_file,sep=',',comment='"')
df.head()

,date_dy,co2_dy,date_wk,co2_wk,date_mn,co2_mn,date_mn_fit,co2_mn_fit
0,2021.152055,415.96,2021.156164,416.45,2021.202740,417.16,2021.202740,417.04
1,2021.154795,416.16,2021.175342,417.56,2021.287671,418.24,2021.287671,418.44
2,2021.157534,416.26,2021.194521,416.54,2021.369863,418.95,2021.369863,419.21
3,2021.163014,417.23,2021.213699,418.00,2021.454795,418.70,2021.454795,418.55
4,2021.165753,416.64,2021.232877,416.43,2021.536986,416.65,2021.536986,416.94


### Break into dataframes for each time frequency

In [6]:
df_daily = df[['date_dy', 'co2_dy']].copy()
df_wk = df[['date_wk', 'co2_wk']].copy()
df_mn = df[['date_mn', 'co2_mn']].copy()
df_mn_fit = df[['date_mn_fit', 'co2_mn_fit']].copy()


### Remove rows with fill values NaN

In [7]:
df_daily = df_daily[df_daily['co2_dy'].notnull()]
df_weekly = df_wk[df_wk['co2_wk'].notnull()]
df_monthly = df_mn[df_mn['co2_mn'].notnull()]
df_monthly_fit = df_mn_fit[df_mn_fit['co2_mn_fit'].notnull()]

print(df_monthly.tail())
print(df_monthly_fit.tail())

        date_mn  co2_mn
19  2022.789041  415.31
20  2022.873973  417.04
21  2022.956164  418.58
22  2023.041096  419.24
23  2023.126027  420.33
    date_mn_fit  co2_mn_fit
19  2022.789041      415.17
20  2022.873973      416.71
21  2022.956164      418.21
22  2023.041096      419.37
23  2023.126027      420.24


## Extract out Maunakea (MKO) data

Needed to differentiate MKO data when plotting

### Get start and end decimal dates of MKO data (12/1/2022 to 3/8/2023)

In [8]:
mauna_kea_start_date = datetime(2022, 12, 1)

year = mauna_kea_start_date.year
boy = datetime(year, 1, 1)
eoy = datetime(year + 1, 1, 1)
mko_decimal_start_date = year + ((mauna_kea_start_date - boy).total_seconds() / ((eoy - boy).total_seconds()))

mauna_kea_end_date = datetime(2023, 3, 8)

year = mauna_kea_end_date.year
boy = datetime(year, 1, 1)
eoy = datetime(year + 1, 1, 1)
mko_decimal_end_date = year + ((mauna_kea_end_date - boy).total_seconds() / ((eoy - boy).total_seconds()))



### Filter data to get MKO data

In [9]:
df_daily_mko = df_daily[(df_daily['date_dy'] >= mko_decimal_start_date) & (df_daily['date_dy'] <= mko_decimal_end_date)]

df_weekly_mko = df_weekly[(df_weekly['date_wk'] >= mko_decimal_start_date) & (df_weekly['date_wk'] <= mko_decimal_end_date)]

df_monthly_mko = df_monthly[(df_monthly['date_mn'] >= mko_decimal_start_date) & (df_monthly['date_mn'] <= mko_decimal_end_date)]

df_monthly_fit_mko = df_monthly_fit[(df_monthly_fit['date_mn_fit'] >= mko_decimal_start_date) & (df_monthly_fit['date_mn_fit'] <= mko_decimal_end_date)]

### Convert to numpy arrays for plotting

In [10]:
date_daily = df_daily['date_dy'].to_numpy()
co2_daily = df_daily['co2_dy'].to_numpy()

date_weekly = df_weekly['date_wk'].to_numpy()
co2_weekly = df_weekly['co2_wk'].to_numpy()

date_monthly = df_monthly['date_mn'].to_numpy()
co2_monthly = df_monthly['co2_mn'].to_numpy()

date_monthly_fit = df_monthly_fit['date_mn_fit'].to_numpy()
co2_monthly_fit = df_monthly_fit['co2_mn_fit'].to_numpy()

date_daily_mko = df_daily_mko['date_dy'].to_numpy()
co2_daily_mko = df_daily_mko['co2_dy'].to_numpy()

date_weekly_mko = df_weekly_mko['date_wk'].to_numpy()
co2_weekly_mko = df_weekly_mko['co2_wk'].to_numpy()

date_monthly_mko = df_monthly_mko['date_mn'].to_numpy()
co2_monthly_mko = df_monthly_mko['co2_mn'].to_numpy()

date_monthly_fit_mko = df_monthly_fit_mko['date_mn_fit'].to_numpy()
co2_monthly_fit_mko = df_monthly_fit_mko['co2_mn_fit'].to_numpy()

## **Define accessory functions**

## Functions to convert dates into various formats
Used for plot limits and labeling

### Function to convert a datetime to a float

In [11]:
# Function to convert datetime to a float
#  https://stackoverflow.com/questions/19305991/convert-fractional-years-to-a-real-date-in-python
def dt2t(adatetime):
    """
    Convert adatetime into a float. The integer part of the float should
    represent the year.
    Order should be preserved. If adate<bdate, then d2t(adate)<d2t(bdate)
    time distances should be preserved: If bdate-adate=ddate-cdate then
    dt2t(bdate)-dt2t(adate) = dt2t(ddate)-dt2t(cdate)
    """
    year = adatetime.year
    boy = datetime(year, 1, 1)
    eoy = datetime(year + 1, 1, 1)
    return year + ((adatetime - boy).total_seconds() / ((eoy - boy).total_seconds()))

### Function to convert a float to a datetime

In [12]:
#  https://stackoverflow.com/questions/19305991/convert-fractional-years-to-a-real-date-in-python
def t2dt(atime):
    """
    Convert atime (a float) to DT.datetime
    This is the inverse of dt2t.
    assert dt2t(t2dt(atime)) == atime
    """
    year = int(atime)
    remainder = atime - year
    boy = datetime(year, 1, 1)
    eoy = datetime(year + 1, 1, 1)
    seconds = remainder * (eoy - boy).total_seconds()
    return boy + timedelta(seconds=seconds)

### Function to create various date versions for today's date

In [13]:
def get_todays_date_variations():
    
    now = datetime.now()
    todays_decimal =  dt2t(now)
    
    today = date.today()

    todays_day = today.day
    todays_month = today.strftime("%B")
    todays_year = today.year
    todays_date_moyr = today.strftime("%B %Y")
    todays_date_modyyr = f"{todays_month} {todays_day}, {todays_year}"

    return todays_date_moyr, todays_date_modyyr, todays_decimal, todays_year

### Function to get the end date of data

In [14]:
def get_data_end_date(date_data):
    last_date = np.max(date_data)
    print(last_date)
    last_date_datetime = t2dt(last_date)
    
    # Convert date format to month_name day, year
    day = last_date_datetime.day
    month = last_date_datetime.strftime("%B")
    year = last_date_datetime.year
    date_modyyr = f"{month} {day}, {year}"
    return date_modyyr

<a name="functions-to-modify-graphic"></a>
## **Define functions that can modify the graphic**

### Function to create gradient under curve

In [15]:
# Gradient function modified from the original function in
# the answer from https://stackoverflow.com/questions/29321835/is-it-possible-to-get-color-gradients-under-curve-in-matplotlib

# Modified to add gradient below curve and have a bottom alpha
# The color under the plot line varies as a gradient from the line to the
# bottom axis.

# Modify arguments of the original function to include limits of the 
# gradient under the curve, the transparency and gradient color
# def gradient_fill(x, y, fill_color=None, ax=None, **kwargs):
def gradient_fill(x, y, fill_color='#FFFFFF', xmin=None, ymin=None, xmax=None, 
                  ymax=None, alpha_bottom=None, ax=None, alpha=1, zorder=1, **kwargs):
    """
    Plot a line with a linear alpha gradient filled beneath it.

    Parameters
    ----------
    x, y : array-like
        The data values of the line.
    fill_color : a matplotlib color specifier (string, tuple) or None
        The color for the fill. If None, the color of the line will be used.
    ax : a matplotlib Axes instance
        The axes to plot on. If None, the current pyplot axes will be used.
    Additional arguments are passed on to matplotlib's ``plot`` function.

    Returns
    -------
    line : a Line2D instance
        The line plotted.
    im : an AxesImage instance
        The transparent gradient clipped to just the area beneath the curve.
    """
    if ax is None:
        ax = plt.gca()

    # Only want the gradient to show and not the line defining the gradient shape
    # so change the original function
    #line, = ax.plot(x, y, **kwargs)
    #if fill_color is None:
    #    fill_color = line.get_color()

    if alpha_bottom is None:
        alpha_bottom = 0

    if xmin is None:
        xmin = x.min()

    if ymin is None:
        ymin = y.min()
        
    if xmax is None:
        xmax = x.max()

    if ymax is None:
        ymax = y.max()
        
    if zorder is None:
        zorder = 1
        
    # zorder in alpha of the gradient are now set in the function arguments,
    # so modify this portion of the original function
    #zorder = line.get_zorder()
    
    #alpha = line.get_alpha()
    #alpha = 1.0 if alpha is None else alpha

    z = np.empty((100, 1, 4), dtype=float)

    rgb = mcolors.colorConverter.to_rgb(fill_color)
    z[:, :, :3] = rgb
    #z[:,:,-1] = np.linspace(0, alpha, 100)[:,None]
    z[:, :, -1] = np.linspace(alpha_bottom, alpha, 100)[:, None]

    # Get the gradient limits from the function arguments,
    # so modify the original function
    #xmin, xmax, ymin, ymax = x.min(), x.max(), y.min(), y.max()
    
    im = ax.imshow(z, aspect='auto', extent=[xmin, xmax, ymin, ymax],
                   origin='lower', zorder=zorder)

    xy = np.column_stack([x, y])
    xy = np.vstack([[xmin, ymin], xy, [xmax, ymin], [xmin, ymin]])
    clip_path = Polygon(xy, facecolor='none', edgecolor='none', closed=True)
    ax.add_patch(clip_path)
    im.set_clip_path(clip_path)
    
    ax.autoscale(True)
    
    # Modify original function since there is no need 
    # to return the line and im objects
    #return line, im

### Function to set global fonts and linewidth properties

In [16]:
def set_matplotlib_properties():
    
    # Set default properties for matplotlib
    
    # Reset rcparams in case modified defaults in another notebook during same session 
    plt.rcParams.update(plt.rcParamsDefault)

    plt.rcParams.update({'axes.linewidth':1.5})
    
    plt.rcParams.update({
        "text.usetex": False,
        "font.family": "sans-serif",
        "font.weight":  "normal",
        "font.sans-serif": ["Arial", "Tahoma", "Helvetica","FreeSans", "NimbusSans", "LiberationSans","DejaVu Sans"],
        "mathtext.default":'regular',
        "mathtext.fontset": "dejavusans"
    })
        
    # http://phyletica.org/matplotlib-fonts/
    # This causes matplotlib to use Type 42 (a.k.a. TrueType) fonts 
    # for PostScript and PDF files. This allows you to avoid Type 3 fonts.
    # Turning on usetex also works
    # Needed on Ubuntu for pdf fonts 
    plt.rcParams.update({
      'pdf.fonttype': 42,
        'ps.fonttype': 42 
    })

### Function to set axes properties of the graphic

In [17]:
def set_plot_props(ax, fig, xmin, xmax, ymin, ymax, xlabel, ylabel):
    
    # ---------------------------------
    # Plot properties for website plots
    # ---------------------------------

    # Allow room at top for the 2 titles
    fig.subplots_adjust(top=0.85)

    ax.tick_params(which='both', bottom=True, top=True, left=True, right=True)

    ax.tick_params(axis='x', labelsize=10, pad=5)
    ax.tick_params(axis='y', labelsize=16)

    ax.tick_params(axis='x', which='major', direction='in', length=6, width=1)
    ax.tick_params(axis='y', which='major', direction='in', length=8, width=1)

    tick_spacing = 5
    ax.yaxis.set_major_locator(ticker.MultipleLocator(tick_spacing))
    
    labels = ax.get_xticklabels() + ax.get_yticklabels()
    for label in labels: label.set_fontweight('bold')  

    ax.tick_params(which='minor', direction='in', length=4)
    ax.yaxis.set_minor_locator(AutoMinorLocator(5))
    
    ax.set_xlabel(xlabel, fontweight='bold', fontsize=18, labelpad=5)
    ax.set_ylabel(ylabel, fontweight='bold', fontsize=18, labelpad=5)

    # Set axes limits last 
    # If did before, setting xtick labels past xmin & xmax would have
    # extended the plot limits
    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)

### Function to create text tick labels

In [18]:
def create_xtick_labels(ax, xmin, xmax):
 
    # Have ticks at the first of the month and then move the tick label
    # to position at the start of this tick mark

    # Set tick marks to 1st of each month
    start_date = datetime(xmin.year, xmin.month, 1, 0, 0)
    end_date = datetime(xmax.year, xmax.month, 1, 0, 0)

    date_ticks_dt = []
    current=start_date
    while current <= end_date:
        date_ticks_dt.append(current)
        current += relativedelta(months=1)

    date_tick_labels = [x.strftime("%b") for x in date_ticks_dt]

    date_tick_decimal = [dt2t(x) for x in date_ticks_dt]
    
    plt.xticks(date_tick_decimal, date_tick_labels)

    # Move tick labels so left aligned with tick mark
    # to represent the month as starting at the tick mark
    # and not meaning the middle of the month.
    for tick in ax.xaxis.get_majorticklabels():
        tick.set_horizontalalignment("left")
        
    # Place the starting and ending years below the xaxis tick labels
    start_year = xmin.year
    label_text = start_year    
    ax.annotate(label_text, xy=(0, -0.06), xycoords='axes fraction', fontsize=10,
                horizontalalignment='left', verticalalignment='top', fontweight='bold')

    end_year = xmax.year
    label_text = end_year
    ax.annotate(label_text, xy=(0.97, -0.06), xycoords='axes fraction', fontsize=10,
                horizontalalignment='left', verticalalignment='top', fontweight='bold')

    ax.tick_params(axis='x', pad=7)

### Function to place the titles

In [19]:
def add_plot_title(ax, title1, title2):

    ax.annotate(title1, xy=(0, 1.15), xycoords='axes fraction', fontsize=14,
                horizontalalignment='left', verticalalignment='top', fontweight="normal")
    ax.annotate(title2, xy=(0, 1.07), xycoords='axes fraction', fontsize=18,
                horizontalalignment='left', verticalalignment='top', fontweight = 'bold')

### Function to add an inset label

In [20]:
def add_inset_label(ax, label_start, today):
    label_text = f"{label_start} ending {today}"
    ax.annotate(label_text, xy=(0.03, 0.9), xycoords='axes fraction', fontsize=12,
                horizontalalignment='left', verticalalignment='top')

In [21]:
def add_inset_label_mko(ax, label_text):
    blue = '#1e47b0'
    ax.annotate(label_text, xy=(0.03, 0.8), xycoords='axes fraction', fontsize=12,
                horizontalalignment='left', verticalalignment='top', fontweight="bold", color=blue)

### Function to add legend

In [22]:
def add_legend_labels(ax):

    legend_1 = 'Daily average'
    legend_2 = 'Weekly average'
    legend_3 = 'Monthly average'
    
    legend_properties = {'weight':'bold'}

    black_dot = mlines.Line2D([], [], marker = 'o', color='black',markersize=4, 
             markerfacecolor='black', markeredgecolor='black', markeredgewidth=0, linestyle='None')

    black_filled_circle = mlines.Line2D([], [], marker = 'o', color='black',markersize=8, 
             markerfacecolor='black', markeredgecolor='black', markeredgewidth=0, linestyle='None')

    black_open_circle = mlines.Line2D([], [], marker = 'o', color='black',markersize=10, 
             markerfacecolor='none', markeredgecolor='black', markeredgewidth=1.5, linestyle='None')
    
    # Use upper right for legend
    plt.rc('legend',fontsize=12, loc='upper right')
    ax.legend([black_dot, black_filled_circle, black_open_circle], 
              [legend_1, legend_2, legend_3],frameon=False,borderaxespad=1,
              labelspacing=0.2, prop=legend_properties)


### Function to add the SIO logo

In [23]:
def add_sio_logo(fig, logo_file, xpos, ypos):

    # Convert the logo svg file to a png file with the
    # given scale and dpi
    logo_png = 'logo.png'
    svg2png(url=logo_file, write_to=logo_png, scale=10, dpi=300)

    logo = mpimg.imread(logo_png)

    fig.add_axes([xpos, ypos, 0.2, 0.2], anchor='SE', zorder=1)

    plt.imshow(logo)
    plt.axis('off')

    # Delete the logo png version
    os.remove(logo_png)


### Function to save the graphic

In [24]:
def save_graphic(fig, pdf_file, png_file):

    fig_width, fig_height = fig.get_size_inches()

    # For pdf
    width_in = 10
    height_in = (fig_height/fig_width) * width_in

    fig.set_size_inches(width_in, height_in)

    # Save to a high dpi so that logo png file has a high resolution
    fig.savefig(pdf_file, facecolor='w', edgecolor='w',
                orientation='landscape', format=None,
                transparent=False, bbox_inches='tight', dpi=600)

    # For png
    # Want a png of height 500px

    # First save figure as a png and then scale to size needed

    png_dpi = 300

    fig.savefig(png_file, facecolor='w', edgecolor='w',
                orientation='landscape', dpi=png_dpi, bbox_inches='tight')

    # Now resize the image to be width 1000px for the
    # keelingcurve website
    img = Image.open(png_file)
    img_width, img_height = img.size

    # width_px = 1000
    # img_scale = (width_px / img_width)
    # height_px = int((float(img_height) * float(img_scale)))

    height_px = 500
    img_scale = (height_px / img_height)
    width_px = int((float(img_width) * float(img_scale)))

    #img = img.resize((width_px, height_px), Image.Resampling.LANCZOS)
    img = img.resize((width_px, height_px), Image.ANTIALIAS)
    img.save(png_file)


### Function to create a Context Manager

This allows a message to display in a cell while a function is running.
Usefull for indicating a function is running.

In [25]:
class ShowProgressContextManagerWidgets:

    def __init__(self, out, command=''):
        self.out = out
        self.command = command
        
    def __enter__(self):
        css_style = "<style>span.start { margin:0; padding: 0; color: blue; }</style>"
        html_str = f'<span class="start">Running {self.command}...</span>'

        self.out.clear_output()
        
        with self.out:
            display(HTML(css_style))
            display(HTML(html_str))

        
    def __exit__(self, exc_type, exc_value, exc_tb):
        
        css_style = "<style>span.end { margin: 0; padding: 0; color: green; }</style>"
        html_str = '<span class="end">Finished</span>'

        self.out.clear_output()

        with self.out:
            display(HTML(css_style)) 
            display(HTML(html_str))


# **Steps to create the graphic**

### Change any properties in the section [Functions to modify the graphic](#functions-to-modify-graphic)

### Comment out any function in the overall function [create_graphic](#function-to-create-the-graphic) to remove an element from the final graphic

Comment out a line by adding a # symbol in front of a statement


## **Set properties to use for matplotlib (plotting engine)**

In [26]:
set_matplotlib_properties()

## **Set plot limits and labeling**

### Set titles and axes labels

In [27]:
xlabel = ''
ylabel = "$\mathregular{CO}\\bf{_2}$" + " Concentration (ppm)"

title1 = ""
title2 = "Carbon dioxide concentration at Mauna Loa Observatory*"

### Set yaxis min and max limits

In [28]:
ymin = 405
ymax = 430

### Set xaxis min and max limits

In [29]:
now = datetime.now()
time_ago = now + relativedelta(years=-2)

xmin = time_ago
xmax = now

# Need decimal range for plotting gradiant since
# plotting the gradiant won't work with datetime values
xmin_dt = datetime(year=xmin.year, month=xmin.month, day=xmin.day)
xmin_dec = dt2t(xmin_dt)

xmax_dt = datetime(year=xmax.year, month=xmax.month, day=xmax.day)
xmax_dec = dt2t(xmax_dt)


<a name="function-to-create-the-graphic"></a>
## **Function to create the graphic**

In [30]:
def create_graphic(msg_out, graphic_out, xmin, xmax, ymin, ymax, xlabel, ylabel, title1, title2, date_daily, co2_daily, date_weekly, co2_weekly, date_monthly, co2_monthly, btn):

    msg_out.clear_output()
    graphic_out.clear_output()

    # create a figure container
    fig = plt.figure()

    # Set axes
    # Add padding for exterior plot text
    ax = plt.axes([0.11, 0.12, 0.83, 0.54])

    # ---------------------------------------------------
    # Context manager 
    # 
    # Used to show messages while creating the animiation
    # ---------------------------------------------------

    with ShowProgressContextManagerWidgets(msg_out, 'Create Graphic') as progress:

        # ------------------------
        # Add gradient under curve
        # ------------------------

        # The gradient is a guide to the eye

        # For gradient under all points, extend monthly data fit to last data point

        max_date_daily = np.max(date_daily)
        max_date_weekly = np.max(date_weekly)
        max_date_monthly = np.max(date_monthly)
        max_date_monthly_fit = np.max(date_monthly_fit)

        max_date = max(max_date_daily, max_date_weekly, max_date_monthly)

        x_gradient = date_monthly_fit
        y_gradient = co2_monthly_fit

        x_monthly_extra = []
        y_monthly_extra = []

        x_weekly_extra = []
        y_weekly_extra = []

        x_daily_extra = []
        y_daily_extra = []

        if max_date_monthly > max_date_monthly_fit:

            # Get monthly data after last monthly fit date
            df_monthly_extra = df_monthly[df_monthly['date_mn'] > max_date_monthly_fit].copy()

            x_monthly_extra = df_monthly_extra['date_mn'].to_numpy()
            y_monthly_extra = df_monthly_extra['co2_mn'].to_numpy()

        if max_date_weekly > max_date_monthly:

            # Get weekly data after last monthly date
            df_weekly_extra = df_weekly[df_weekly['date_wk'] > max_date_monthly].copy()

            x_weekly_extra = df_weekly_extra['date_wk'].to_numpy()
            y_weekly_extra = df_weekly_extra['co2_wk'].to_numpy()

        if max_date_daily > max_date_weekly:

            # Get daily data after last weekly date
            df_daily_extra = df_daily[df_daily['date_dy'] > max_date_weekly].copy()

            x_daily_extra = df_daily_extra['date_dy'].to_numpy()
            y_daily_extra = df_daily_extra['co2_dy'].to_numpy()


        x_extra = np.concatenate((x_monthly_extra, x_weekly_extra, x_daily_extra))
        y_extra = np.concatenate((y_monthly_extra, y_weekly_extra, y_daily_extra))

        if x_extra.size:
            # Make a simple spline through y_extra for the gradient as a guide to the eye
            # Add in one point before the extra values

            x_initial = date_monthly[-1]
            y_initial = co2_monthly[-1]

            x_extra = np.insert(x_extra, 0, x_initial, axis=0)
            y_extra = np.insert(y_extra, 0, y_initial, axis=0)

            poly = np.polyfit(x_extra, y_extra, deg=1)

            y_extra_fit = np.polyval(poly, x_extra)
        else:
            y_extra_fit = np.zeros(shape=(1,0))

        x_gradient = np.concatenate((x_gradient, x_extra))
        y_gradient = np.concatenate((y_gradient, y_extra_fit))


        # For gradient max, use last date of data rather than current date
        xmax_gradient = max_date

        #area_color = '#ACCAE6'
        #area_color = '#8CB2E0'
        area_color = '#73a7e6'

        gradient_fill(x_gradient,y_gradient, fill_color=area_color, xmin=xmin_dec, ymin=ymin, 
                xmax=xmax_gradient, ymax=ymax, alpha_bottom=0.1, ax=ax, alpha=1.0)

        # -------------
        # Plot the data
        # -------------

        # Set plotting zorder above the gradient

        ax.plot(date_daily, co2_daily, 'o', color='black',markersize=4, 
                markerfacecolor='black', markeredgecolor='black', markeredgewidth=0, zorder=5)

        ax.plot(date_weekly, co2_weekly, 'o', color='black',markersize=8, 
                markerfacecolor='black', markeredgecolor='black', markeredgewidth=0, zorder=6)

        ax.plot(date_monthly, co2_monthly, 'o', color='black',markersize=10, 
                markerfacecolor='none', markeredgecolor='black', markeredgewidth=1.5, zorder=7)

        # differentiate MKO data
        blue = '#1e47b0'
        ax.plot(date_daily_mko, co2_daily_mko, 'o', color=blue,markersize=4, 
                markerfacecolor=blue, markeredgecolor=blue, markeredgewidth=0, zorder=10)

        ax.plot(date_weekly_mko, co2_weekly_mko, 'o', color=blue,markersize=8, 
                markerfacecolor=blue, markeredgecolor=blue, markeredgewidth=0, zorder=11)

        ax.plot(date_monthly_mko, co2_monthly_mko, 'o', color=blue,markersize=10, 
                markerfacecolor='none', markeredgecolor=blue, markeredgewidth=1.5, zorder=12)

        # -----------------------------------------------------
        # Change xaxis to use date labels and not decimal dates
        # -----------------------------------------------------
        create_xtick_labels(ax, xmin, xmax)

        # -------------------
        # Set plot properties
        # -------------------
        set_plot_props(ax, fig, xmin_dec, xmax_dec, ymin, ymax, xlabel, ylabel)

        # ------------------------------------------------
        # Get todays date in various formats for labeling
        # ------------------------------------------------
        todays_date_moyr, todays_date_modyyr, todays_decimal, todays_year = get_todays_date_variations()

        # ---------------
        # Add plot titles
        # ---------------
        add_plot_title(ax, title1, title2)

        # --------------
        # Add inset text
        # --------------
        add_inset_label(ax, 'Two years', todays_date_modyyr)

        add_inset_label_mko(ax, "*Maunakea data in blue")

        # ----------------
        # Add legend
        # ----------------
        add_legend_labels(ax)

        # --------------------
        # Add SIO logo to plot
        # --------------------
        xpos = 0.715
        ypos = 0.155

        add_sio_logo(fig, logo_file, xpos, ypos)

        # ------------
        # Save graphic
        # ------------
        fig = plt.gcf()

        save_graphic(fig, pdf_file, png_file)


    with graphic_out:

        plt.show()

<a name='button-to-create-and-save-the-graphic'></a>
## **Button to create and save the graphic**

In [31]:
msg_out = widgets.Output()
graphic_out = widgets.Output()

button = widgets.Button(description='Create Graphic', button_style='primary')
button.on_click(functools.partial(create_graphic, msg_out, graphic_out, xmin, xmax, ymin, ymax, xlabel, ylabel, title1, title2, date_daily, co2_daily, date_weekly, co2_weekly, date_monthly, co2_monthly))
display(button)

display(msg_out)
display(graphic_out)

Button(button_style='primary', description='Create Graphic', style=ButtonStyle())

Output()

Output()

## **Button to download PDF & PNG files of the graphic**

The download button only works when the notebook is run with Google Colab

If the notebook is run on a local machine, the graphics are inside the plots directory

In [32]:
def download_files(msg_out, btn):

    msg_out.clear_output()

    try:
        files.download(png_file)
        files.download(pdf_file)
    except:
        with msg_out:
            print("If running notebook in Google Colab:")
            print("Need to create the graphic first.")
            print(f"\nIf running in a python environment, the graphics are in the plots directory.")


msg_out = widgets.Output()

button = widgets.Button(description="Download Graphic Files", button_style="primary")

layout = widgets.Layout(width='auto', height='auto') #set width and height

button = widgets.Button(
    description='Download Graphic Files',
    disabled=False,
    display='flex',
    flex_flow='column',
    align_items='stretch', 
    button_style='primary',
    layout = layout
)   

button.on_click(functools.partial(download_files, msg_out))

display(button)
display(msg_out)

Button(button_style='primary', description='Download Graphic Files', layout=Layout(height='auto', width='auto'…

Output()